In [41]:
from pathlib import Path
import random
import re
import string

from bs4 import BeautifulSoup
from fastwarc.warc import ArchiveIterator, WarcRecordType
from fastwarc.stream_io import GZipStream
from tqdm.auto import tqdm

from cs336_data import extract, gopher

In [2]:
extract.extract_text_from_warc_gz(
    warc_gz_path="../data/scrape/enwiki_urls_sampled_100k.warc.gz",
    out_dir="../data/scrape/warc_text"
)

In [6]:
in_dir = "../data/scrape/warc_text"
out_dir = "../data/scrape/filter_text"
Path(out_dir).mkdir(parents=True, exist_ok=True)
for in_path in tqdm(list(Path(in_dir).iterdir())):
    with open(in_path, "r") as in_f:
        text = in_f.read()
    filter_ = gopher.gopher_quality_filter(text)
    if filter_:
        out_path = f"{out_dir}/{in_path.name}"
        with open(out_path, "w") as out_f:
            out_f.write(text)

  0%|          | 0/121183 [00:00<?, ?it/s]

# negatives

In [8]:
cc_dir = "../data/CC-MAIN-2023-50-warc-filtered"

In [51]:
def html_bytes_to_text(html_bytes):
    soup = BeautifulSoup(html_bytes)
    text = soup.get_text()
    return text

In [52]:
punct_re = re.compile(f"([{re.escape(string.punctuation)}])")
whitespace_re = re.compile(r"\s+")

In [53]:
def normalize_text(text):
    text = punct_re.sub(r" \1 ", text)
    text = text.lower()
    text = whitespace_re.sub(" ", text)
    text = text.strip()
    return text

In [58]:
def preprocess_for_fasttext(html_bytes):
    text = html_bytes_to_text(html_bytes)
    text = normalize_text(text)
    return text

In [59]:
def html_filter(record):
    return record.http_content_type != "text/html"

In [66]:
# num_samples = 100_000
# num_warc_files = 100
num_samples = 10
num_warc_files = 1

In [67]:
count = 0
reservoir = [None for _ in range(num_samples)]
warc_gz_paths = list(Path(cc_dir).iterdir())
warc_gz_paths = warc_gz_paths[:num_warc_files]
for warc_gz_path in tqdm(warc_gz_paths):
    with open(warc_gz_path, "rb") as gz:
        with GZipStream(gz) as stream:
            archive_iterator = ArchiveIterator(
                stream,
                record_types=WarcRecordType.response,
                min_content_length=250,
                max_content_length=500_000,
                func_filter=html_filter,
            )
            for record in archive_iterator:
                count += 1
                r_idx = random.randrange(count)
                if r_idx >= num_samples:
                    continue
                html_bytes = record.reader.read()


  0%|          | 0/1 [00:00<?, ?it/s]

In [62]:
print(reservoir[2])

fotografia e progettazione : workshop in giardino hai un ' attività di giardinaggio ? iscriviti login video , notizie , prodotti dal mondo del giardinaggio iscriviti alla newsletter contenutivideo , news , prodotti argomenti prato attrezzature piante e fiori agrofarmaci outdoor formazione mercato verde sportivo tutte le tags eventi piante e fiori prodotti da giardino manutenzione verde giardinaggio macchine da giardino giardini giardinieri formazione eima tutte le tags tipologia video news prodotti eventi vedi tutti ultimi post mai più senza energia ascolta il podcast aiuole allâ€™ombra e al sole nuovo e usato solo annunci per il giardinaggio communityproduttori , negozi e non solo cerca per località lombardia ( 230 ) veneto ( 115 ) emilia romagna ( 85 ) piemonte ( 71 ) toscana ( 63 ) lazio ( 44 ) sicilia ( 34 ) puglia ( 30 ) friuli venezia giulia ( 26 ) marche ( 24 ) campania ( 23 ) estero ( 23 ) trentino alto adige ( 22 ) umbria ( 20 ) liguria ( 19 ) abruzzo ( 16 ) calabria ( 16 ) sa